In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
n_hidden = 25
lr = 0.01
epochs= 1000

string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)
print(n_letters)

35


In [3]:
#문장이 들어왔을때, 시작토큰과 끝 토큰을 붙이고 문자하나씩 원핫벡터로 변환하는 함수
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list), dtype=int)
    end = np.zeros(shape=len(char_list), dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape= n_letters, dtype=int)
        zero[idx] =1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [4]:
output = string_to_onehot('abc')
print(output)#잘 출력되는지 확인

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


In [5]:
#원핫벡터(벡터하나)를 다시 문자로 바꾸는 함수
def onehot_to_word(onehot_1):
    return char_list[onehot_1.argmax()]

In [6]:
print(torch.tensor(output, dtype=torch.int64))
print(onehot_to_word(torch.tensor(output)[1]))

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
a


In [7]:
#Simple RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN,self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.h2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

In [8]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [9]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j]  #input_shape = (n_letters)
        target = one_hot[j+1]#target_shape = (n_letters)
        
        output, hidden = rnn.forward(input_, hidden) #output_shape = (1, n_letters)
        loss = loss_func(output.view(-1),target)
        total_loss += loss
    
    total_loss.backward()
    optimizer.step()
    
    if i%10 == 0:
        print(total_loss)
        
        
        

tensor(3.8392, grad_fn=<AddBackward0>)
tensor(1.6709, grad_fn=<AddBackward0>)
tensor(1.3423, grad_fn=<AddBackward0>)
tensor(1.1048, grad_fn=<AddBackward0>)
tensor(0.9319, grad_fn=<AddBackward0>)
tensor(0.7785, grad_fn=<AddBackward0>)
tensor(0.7574, grad_fn=<AddBackward0>)
tensor(0.6042, grad_fn=<AddBackward0>)
tensor(0.5602, grad_fn=<AddBackward0>)
tensor(0.4887, grad_fn=<AddBackward0>)
tensor(0.4497, grad_fn=<AddBackward0>)
tensor(0.4411, grad_fn=<AddBackward0>)
tensor(0.3991, grad_fn=<AddBackward0>)
tensor(0.3726, grad_fn=<AddBackward0>)
tensor(0.3509, grad_fn=<AddBackward0>)
tensor(0.3351, grad_fn=<AddBackward0>)
tensor(0.3232, grad_fn=<AddBackward0>)
tensor(0.3078, grad_fn=<AddBackward0>)
tensor(0.2994, grad_fn=<AddBackward0>)
tensor(0.3011, grad_fn=<AddBackward0>)
tensor(0.3079, grad_fn=<AddBackward0>)
tensor(0.2825, grad_fn=<AddBackward0>)
tensor(0.3080, grad_fn=<AddBackward0>)
tensor(0.2757, grad_fn=<AddBackward0>)
tensor(0.2628, grad_fn=<AddBackward0>)
tensor(0.3022, grad_fn=<A

In [10]:
start = torch.zeros(1,len(char_list))
start[:, -2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output)
        input_ = output
        
print(output_string)  #뒤로 갈수록 잘 학습하지 못한다.      


hellcmeite teiteieeieciectecteiteiteieciectecteiteiteieeiectectectei


In [11]:
#LSTM
rnn = nn.LSTM(input_size=3, hidden_size=5, num_layers=2,batch_first=True)#num_layers는 lstm을 몇 층으로 쌓을 것인지 나타냄
# 입력의 형태는 (seq_len, batch, input_size), batch_first=True라 하면, batch가 가장 먼저 옴.
input_ = torch.randn(3, 5, 3)#여기서는 (batch, seq_len, input_size)
# hidden state, cell state의 형태는 (num_layers * num_directions, batch, hidden_size)
h0 = torch.randn(2, 3, 5)
c0 = torch.randn(2, 3, 5)
# LSTM에 입력을 전달할때는 input, (h_0, c_0) 처럼 상태를 튜플로 묶어서 전달.
output, (hidden_state, cell_state) = rnn(input_, (h0, c0))
print(output.size(),hidden_state.size(),cell_state.size())


torch.Size([3, 5, 5]) torch.Size([2, 3, 5]) torch.Size([2, 3, 5])


In [12]:
# 하이퍼파라미터 설정
# 이번코드는 배치사이즈가 1보다 큰 경우에 대해 만들었습니다.
batch_size = 5

# seq_len는 바꿔도 학습은 되지만 테스트시 편의성을 위해 1로 설정했습니다.
seq_len = 1

# num_layers는 자유롭게 바꿀 수 있습니다.
num_layers = 3
input_size = n_letters #35
hidden_size = 35
lr = 0.01
num_epochs = 1000

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

print(one_hot.size())

torch.Size([70, 35])


In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
    def forward(self,input,hidden,cell):
        output,(hidden,cell) = self.lstm(input,(hidden,cell))
        
        return output,hidden,cell
    
    def init_hidden_cell(self, batch_size):
        hidden = torch.zeros(num_layers, batch_size, hidden_size) 
        cell = torch.zeros(num_layers, batch_size, hidden_size)
        return hidden,cell
    
lstm = RNN(input_size, hidden_size, num_layers)

In [14]:
# Loss function & Optimizer

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=lr)

In [15]:
j=0
input_data = one_hot[j:j+batch_size].view(batch_size,seq_len,input_size)
print(input_data.size())

hidden,cell = lstm.init_hidden_cell(batch_size)
print(hidden.size(),cell.size())

output,hidden,cell = lstm(input_data,hidden,cell)
print(output.size(),hidden.size(),cell.size())
#output = (batch, seq_len, hidden_size)
#hidden, cell = (num_layer*num_direction, batch, hidden_size)

torch.Size([5, 1, 35])
torch.Size([3, 5, 35]) torch.Size([3, 5, 35])
torch.Size([5, 1, 35]) torch.Size([3, 5, 35]) torch.Size([3, 5, 35])


In [16]:
unroll_len = one_hot.size()[0]//seq_len -1#75//1 -1 = 74 끝 토큰은 입력으로 훈련시킬 필요없으므로
for i in range(num_epochs):
    optimizer.zero_grad()
    hidden,cell = lstm.init_hidden_cell(batch_size)
    
    loss = 0
    for j in range(unroll_len-batch_size+1):#74-3+1= 72-> index 71,72,73 벡터를 batch로 쌓는다.
        
        # batch size에 맞게 one-hot 벡터를 스택 합니다.
        # 예를 들어 batch size가 3이면 pytorch에서 pyt를 one-hot 벡터로 바꿔서 쌓고
        # 목표값으로 yto를 one-hot 벡터로 바꿔서 쌓는 과정입니다.
        input_data = torch.stack([one_hot[j+k:j+k+seq_len] for k in range(batch_size)],dim=0)
        #input_data = (batch, seq_len, input_size) = ()
        label = torch.stack([one_hot[j+k+1:j+k+seq_len+1] for k in range(batch_size)],dim=0)
        #output_data = (batch,, seq_len, hidden_size)
        input_data = input_data
        label = label
        
        output, hidden, cell = lstm(input_data,hidden,cell)
        loss += loss_func(output.view(1,-1),label.view(1,-1))
        
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(loss)

tensor(2.1304, grad_fn=<AddBackward0>)
tensor(1.6977, grad_fn=<AddBackward0>)
tensor(1.6031, grad_fn=<AddBackward0>)
tensor(1.4696, grad_fn=<AddBackward0>)
tensor(1.2395, grad_fn=<AddBackward0>)
tensor(0.8986, grad_fn=<AddBackward0>)
tensor(0.5138, grad_fn=<AddBackward0>)
tensor(0.2935, grad_fn=<AddBackward0>)
tensor(0.1465, grad_fn=<AddBackward0>)
tensor(0.0893, grad_fn=<AddBackward0>)
tensor(0.0627, grad_fn=<AddBackward0>)
tensor(0.0484, grad_fn=<AddBackward0>)
tensor(0.0390, grad_fn=<AddBackward0>)
tensor(0.0323, grad_fn=<AddBackward0>)
tensor(0.0276, grad_fn=<AddBackward0>)
tensor(0.0242, grad_fn=<AddBackward0>)
tensor(0.0216, grad_fn=<AddBackward0>)
tensor(0.0189, grad_fn=<AddBackward0>)
tensor(0.0170, grad_fn=<AddBackward0>)
tensor(0.0163, grad_fn=<AddBackward0>)
tensor(0.0145, grad_fn=<AddBackward0>)
tensor(0.0131, grad_fn=<AddBackward0>)
tensor(0.0117, grad_fn=<AddBackward0>)
tensor(0.0110, grad_fn=<AddBackward0>)
tensor(0.0106, grad_fn=<AddBackward0>)
tensor(0.0103, grad_fn=<A

In [17]:
start = torch.zeros(1,len(char_list))
start[:, -2] = 1

with torch.no_grad():
    hidden,cell = lstm.init_hidden_cell(1)
    input_ = start.reshape(1,seq_len,input_size)
    output_string = ""
    for i in range(len(string)):
        output,hidden,cell = lstm(input_,hidden,cell)
        output_string += onehot_to_word(output.view(-1))
        input_ = output
        
print(output_string) 

hello pytorch. how long can a rnn cell remember? show me your limit!


## embedding
<정의>
알파벳이나 단어 같은 기본 요소들을 일정한 길이를 가지는 벡터 공간에 투영하는 것이다
특히 단어들을 벡터화하는 것을 word2vec이라고 한다.
<방법>
1. CBOW
-> 주변 단어들로 부터 가운데 들어갈 단어가 나오도록 임베딩하는 방식, 구체적으로는 주변단어를 입력층으로 받고 은닉층을 하나 추가한다. 그리고 임베딩하고자 하는 단어를 출력하도록 모델을 학습시킨다. 최종적으로는 은닉층의 벡터를 임베딩 벡터로 사용한다.

2. Skim-gram
-> CBOW와 반대로, 중심단어로 부터 주변단어가 나오도록 한다. 

임베딩 역시 학습이 가능하다


In [23]:
import unidecode
import string
import random
import re
import time, math

In [24]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [87]:
# import 했던 string에서 출력가능한 문자들을 다 불러옵니다. 
all_characters = string.printable

# 출력가능한 문자들의 개수를 저장해놓습니다.
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [26]:
file = unidecode.unidecode(open('./Ch6_data/input.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


In [60]:
# input.txt의 일부분을 chunk_len 길이만큼 발췌해 오는 함수
def random_chunk(chunk_len):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len 
    return file[start_index:end_index]

chunk_len = 100
print(random_chunk(chunk_len))

Salisbury
Is gone to meet the king, who lately landed
With some few private friends upon this coast.


In [31]:
# 문자열을 받았을때 이를 인덱스의 배열로 바꿔주는 함수입니다.
def char_tensor(string):
    tensor = torch.zeros(len(string)).long() #long type은 int형 자료형이라 생각하면 된다.
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('0BCdef'))

tensor([ 0, 37, 38, 13, 14, 15])


In [32]:
# 랜덤한 텍스트 chunk를 불러와서 이를 입력과 목표값을 바꿔주는 함수입니다.
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [135]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size) 
        #nn.Embedding의 input_size는 사용할 문자의 개수라고 생각하면 된다.
        #인덱스 배열 [0,1,2]를 받으면, 3xembedding_size size로 반환한다.
        self.rnn = nn.RNN(input_size=embedding_size,hidden_size=hidden_size, num_layers=num_layers)
        #self.rnn = nn.GRU(input_size=embedding_size,hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out, hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden
    

In [136]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [137]:
model = RNN(input_size=n_characters,embedding_size = embedding_size, hidden_size=hidden_size,
            output_size=n_characters, num_layers=2).to(device)

In [141]:
inp = char_tensor("A").to(device)
hidden = model.init_hidden().to(device)
out,hidden = model(inp, hidden)
print(out.shape, hidden.shape)

torch.Size([1, 100]) torch.Size([2, 1, 100])


In [142]:
chunk_len = 100
total = char_tensor(random_chunk(chunk_len))
print(total.view(-1,1))

tensor([[94],
        [12],
        [17],
        [18],
        [21],
        [13],
        [27],
        [14],
        [23],
        [94],
        [24],
        [27],
        [94],
        [17],
        [18],
        [28],
        [94],
        [32],
        [18],
        [15],
        [14],
        [68],
        [28],
        [94],
        [10],
        [21],
        [21],
        [18],
        [14],
        [28],
        [96],
        [55],
        [17],
        [18],
        [28],
        [94],
        [18],
        [28],
        [94],
        [29],
        [17],
        [14],
        [94],
        [13],
        [10],
        [34],
        [94],
        [32],
        [17],
        [14],
        [27],
        [14],
        [18],
        [23],
        [94],
        [44],
        [94],
        [32],
        [18],
        [28],
        [17],
        [68],
        [13],
        [94],
        [29],
        [24],
        [94],
        [15],
        [10],
        [21],
        [21],
      

In [143]:
# Loss function & Optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [149]:
# 임의의 문자(start_str)로 시작하는 길이 200짜리 모방 글을 생성하는 코드입니다.
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden().to(device)
    x = inp.to(device)

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)
        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 만약 max 값만 쓰는 경우에
        # 생성되는 텍스트가 다 the the the the the 이런식으로 나오기 때문입니다.
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 랜덤하게 다음 글자를 뽑아내는 방식으로 자연스러운 텍스트를 생성해냅니다.
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        
        print(predicted_char,end="")

        x = char_tensor(predicted_char).to(device)

In [145]:
for i in range(num_epochs):
    total = char_tensor(random_chunk(chunk_len))
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden().to(device)
    
    loss = torch.tensor([0]).type(torch.FloatTensor).to(device)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x = inp[j].to(device)
        y_ = label[j].unsqueeze(0).type(torch.LongTensor).to(device)
        out,hidden = model(x, hidden)
        loss += loss_func(out,y_)
    
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([4.5736], device='cuda:0', grad_fn=<DivBackward0>) 

j89	X*Nt2W=qspoP$!\z`XBn^z<])xoxcdU<9sn#?*:9=hd,w%='[i*T?).xa{_whDT1jH"(AwEwNj	u//5tq.QA}tsc$2:)8q3isr$86R1-?$@x(}q+~>^(_q.zfLR-g(>NL]QWSdSI6kS,@%L(lqa
[G97NK}#`q"
 zAC?[jyB^ip`!R	4Ahpw_

 tensor([2.5167], device='cuda:0', grad_fn=<DivBackward0>) 

bbd wing the aon by letme ahe broee sauy heI be hy inE ar ere sir, polt efrsetin er vestria no sit thet nor ginl there rouwt torees dou te hour wehivos ond an ingnothin yorgy moireils, be tit rone weri

 tensor([2.3652], device='cuda:0', grad_fn=<DivBackward0>) 

befren; bere git hy mee,
Bapein. Ksele sat en heas ciirg witis sals lienfote'd as case sorreies tiund, liten nolsll it
Anpeard sire saneat: hind;
Wath lore thea te stay wethe here of aold, od sat hane 

 tensor([2.3258], device='cuda:0', grad_fn=<DivBackward0>) 

be l fryere, the vove is sill her orstivime sill of her thead!
Dnd dor woet as nom heJ me poud some hed thabe hit shous and feed a cond ca harl the af youghur

In [227]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size) 
        #nn.Embedding의 input_size는 사용할 문자의 개수라고 생각하면 된다.
        #인덱스 배열 [0,1,2]를 받으면, 3xembedding_size size로 반환한다.
        self.rnn = nn.LSTM(input_size=embedding_size,hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        return out, hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden, cell

In [228]:
lstm = RNN(input_size=n_characters,embedding_size = embedding_size, hidden_size=hidden_size,
            output_size=n_characters, num_layers=2).to(device)

In [229]:
# Loss function & Optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=lr)

In [230]:
# 임의의 문자(start_str)로 시작하는 길이 200짜리 모방 글을 생성하는 코드입니다.
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden, cell = lstm.init_hidden()
    hidden = hidden.to(device)
    cell = cell.to(device)
    x = inp.to(device)

    print(start_str,end="")
    for i in range(200):
        output,hidden = lstm(x,hidden,cell)
        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 만약 max 값만 쓰는 경우에
        # 생성되는 텍스트가 다 the the the the the 이런식으로 나오기 때문입니다.
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 랜덤하게 다음 글자를 뽑아내는 방식으로 자연스러운 텍스트를 생성해냅니다.
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        
        print(predicted_char,end="")

        x = char_tensor(predicted_char).to(device)

In [231]:
for i in range(num_epochs):
    total = char_tensor(random_chunk(chunk_len))
    inp = total[:-1]
    label = total[1:]
    hidden, cell = lstm.init_hidden()
    hidden = hidden.to(device)
    cell = cell.to(device)
    loss = torch.tensor([0]).type(torch.FloatTensor).to(device)
    
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x = inp[j].to(device)
        y_ = label[j].unsqueeze(0).type(torch.LongTensor).to(device)
        out,hidden = lstm(x, hidden, cell)
        loss += loss_func(out,y_)
    
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([4.5753], device='cuda:0', grad_fn=<DivBackward0>) 

b\a3rf%ZmX=_X DA7}hwi!lu`F-q-clf<O	KE6)%,K]oD`Aa{2hL(#(1W[#6>5k)jOhK?N{-]#B8;as$&A?Cn/3QBE(.2hvWQYWd}|PK6qsq1Z"^pc;D] #U1I Y"+*)SMOD4WMK5C,-\DTF	|r`"4Bx0<0j9m33D@S2&kpZC1!2J>\|C9VC\/;3!n04n5GJ16ul
^k1

 tensor([2.3813], device='cuda:0', grad_fn=<DivBackward0>) 

ber yi: f thher wer in yod, hfo hinnF
H hen oin h.dletlonn I's thaurrof ire doo ptuur ffer himef ond hher conritrtref atho ro hese boun soverimons Mon aln or, ices bhe pon hur s ror
Nosd thns oner tre


 tensor([2.6588], device='cuda:0', grad_fn=<DivBackward0>) 

br adr par id wo hurle cacons anj kes ougou thod an.
ind amor fifdee' in'linl vord hir of co! I reme sol me.
Inf doy yas ink.
Oo hoe won, ar souit doo oe Ir ans soonf ipir yhy thob-I thas tfoi.
Or or a

 tensor([2.1209], device='cuda:0', grad_fn=<DivBackward0>) 

by hannife fisis get I doeh sant le pon sen kinsire
Mlome lo; whh sice don ecour I ba efote,
Four he thet fou con Powr yound uw he the ipes, bupfe